In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
import os
HOME = os.getcwd()
%cd {HOME}
!git clone https://github.com/ifzhang/ByteTrack.git
%cd {HOME}/ByteTrack

# workaround related to https://github.com/roboflow/notebooks/issues/80
!sed -i 's/onnx==1.8.1/onnx==1.9.0/g' requirements.txt

!pip3 install -q -r requirements.txt
!python3 setup.py -q develop
!pip install -q cython_bbox
!pip install -q onemetric
# workaround related to https://github.com/roboflow/notebooks/issues/112 and https://github.com/roboflow/notebooks/issues/106
!pip install -q loguru lap thop

from IPython import display
display.clear_output()


import sys
sys.path.append(f"{HOME}/ByteTrack")


import yolox
print("yolox.__version__:", yolox.__version__)

!pip install supervision==0.1.0
display.clear_output()

import supervision
print("supervision.__version__:", supervision.__version__)

from supervision.tools.detections import Detections, BoxAnnotator
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import moviepy.video.fx.all as vfx
import pandas as pd
from PIL import Image
import csv
import math
from ultralytics  import YOLO
import time
from keras.models import load_model
from typing import List
from yolox.tracker.byte_tracker import BYTETracker, STrack
from onemetric.cv.utils.iou import box_iou_batch
from dataclasses import dataclass

supervision.__version__: 0.1.0


In [3]:
from PIL import Image
import csv

def divide_image(img, rows, cols, output_directory="/content/"):
    # Obtenir les dimensions de l'image
    img_width, img_height = img.size

    # Calculer les dimensions d'une zone
    zone_width = img_width // cols
    zone_height = img_height // rows

    # Diviser l'image en zones
    with open(f"{output_directory}/zones_info.csv", "w", newline="") as csvfile:
        fieldnames = ["zone_id", "x0", "y0", "x1", "y1"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in range(rows):
            for col in range(cols):
                # Coordonnées du coin supérieur gauche de la zone
                x0 = col * zone_width
                y0 = row * zone_height

                # Coordonnées du coin inférieur droit de la zone
                x1 = x0 + zone_width
                y1 = y0 + zone_height

                # Extraire la zone de l'image
                zone_img = img.crop((x0, y0, x1, y1))

                # Sauvegarder la zone dans un fichier
                zone_img.save(f"{output_directory}/zone_{row}_{col}.png")

                # Écrire les informations de la zone dans le fichier CSV
                writer.writerow({"zone_id": f"zone_{row}_{col}", "x0": x0, "y0": y0, "x1": x1, "y1": y1})

In [4]:
# YOLO class definition is assumed to be defined elsewhere in the code.

def detect_objects_in_zone(zone_image_path, model_path, conf_val, output_directory, iou, agnostic_nms):
    # Create an instance of the YOLO model using the specified model path
    model = YOLO(model_path)

    # Read the input image and convert it to a NumPy array in BGR format
    frame = Image.open(zone_image_path)
    frame_np = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)

    # Perform object detection using the YOLO model on the input image
    result = model.predict(frame, conf=conf_val, classes = 0, iou = iou,agnostic_nms = agnostic_nms, show=False, verbose=False)

    # Count the number of swimmers detected
    boxes = result[0].boxes
    num_swimmers = 0
    for box in boxes:
        confidence = box.conf[0]
        confidence = math.ceil(confidence * 100)
        if confidence > 25:
            num_swimmers += 1
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # Draw a rectangle around the detected object on the image
            cv2.rectangle(frame_np, (x1, y1), (x2, y2), (0, 0, 255), 5)

    # Draw text on the image to display the number of swimmers detected
    cv2.putText(frame_np, f'{"num_swimmers :"} {num_swimmers}', (5, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 4)

    # Save the modified image with rectangles and text
    filename = os.path.basename(zone_image_path)
    filename = os.path.splitext(filename)[0]  # Remove the file extension
    new_filename = f"{filename}_{num_swimmers}.png"
    output_path = os.path.join(output_directory, new_filename)
    cv2.imwrite(output_path, frame_np)

    # Return the total number of swimmers detected
    return num_swimmers

In [5]:
@dataclass(frozen=True)
class BYTETrackerArgs:
    """ Ce paramètre contrôle la sensibilité du suivi. Si vous choisissez une valeur plus basse, le tracker sera plus sensible
    et considérera plus d'objets comme correspondances potentielles. Une valeur plus élevée rendra le tracker moins sensible.
    La valeur optimale dépendra de la qualité des données d'entrée, de la vitesse des objets et du degré de variation entre les
    images successives. """
    track_thresh: float = 0.25
    """ Le track buffer définit la durée pendant laquelle une piste peut être perdue avant d'être désactivée. Si vos objets sont
    susceptibles de disparaître temporairement et de réapparaître dans un court laps de temps, vous pourriez envisager d'augmenter
    ce paramètre. Si les objets disparaissent rarement, une valeur plus basse peut être appropriée. """
    track_buffer: int = 300
    """Correspondance Précise vs. Correspondance Lâche :
    Une valeur élevée (par exemple, 0.8) nécessitera que la nouvelle détection soit très similaire à la piste existante pour être
    acceptée comme une correspondance. Cela peut aider à prévenir les fausses correspondances et à maintenir une haute précision,
    mais cela peut aussi rendre le suivi plus restrictif, ce qui pourrait être problématique si les objets subissent des variations
    d'apparence importantes.
    Une valeur plus basse (par exemple, 0.5) permettra des correspondances plus lâches, ce qui peut être utile lorsque les objets
    subissent des variations d'apparence, d'éclairage ou de pose. Cela peut également augmenter la sensibilité du suivi, mais il
    pourrait y avoir plus de risque de fausses correspondances.

    Lorsque l'on parle de "détection" dans le contexte du suivi d'objets, il s'agit généralement de l'identification et de la
    localisation d'un objet spécifique dans une image ou une séquence d'images. Une "piste existante" fait référence à la trajectoire
    ou au chemin suivi par un objet particulier au fil du temps.
    Lorsque l'on dit que "la nouvelle détection doit être très similaire à la piste existante pour être acceptée comme une correspondance",
    cela signifie que pour qu'une nouvelle détection soit considérée comme une correspondance valable avec une piste existante, elle doit
    partager certaines caractéristiques ou propriétés similaires avec cette piste existante. En d'autres termes, la nouvelle détection
    doit ressembler suffisamment à l'objet que nous suivons déjà.
    Cela peut inclure des similitudes dans divers aspects de l'objet, tels que :
    Position : La nouvelle détection doit être proche de l'emplacement prévu de l'objet en fonction de sa trajectoire précédente.
    Apparence : Les caractéristiques visuelles de l'objet doivent être cohérentes avec ce que nous avons observé auparavant. Cela peut
    inclure la forme générale, les couleurs, les textures, etc.
    Mouvement : Le mouvement de la nouvelle détection doit être cohérent avec le mouvement attendu de l'objet, en tenant compte de sa
    vitesse et de sa direction.
    Taille et Échelle : La taille apparente de l'objet dans la nouvelle détection doit être cohérente avec la taille attendue.
    Contexte : L'environnement ou le contexte autour de l'objet doit également être cohérent avec les observations précédentes.


    Caractéristiques des Objets et Variations :
    Si les objets que vous suivez sont très similaires et peu susceptibles de subir des variations significatives, une valeur plus
    élevée pourrait être appropriée.
    Si les objets ont des variations importantes dans leur apparence, une valeur plus basse pourrait être nécessaire pour permettre
    une correspondance réussie malgré ces variations.

    Environnement et Bruit :
    Si votre scène contient du bruit ou des objets perturbateurs qui pourraient générer de fausses détections, une valeur plus
    élevée pourrait aider à filtrer ces correspondances erronées.
    Si vous êtes sûr que votre scène est relativement propre et qu'il y a peu de risque de fausses détections, une valeur plus
    basse pourrait être utilisée pour capturer davantage de correspondances potentielles.

    Évaluation et Expérimentation :
    Il est souvent recommandé de tester différentes valeurs sur des ensembles de données de test ou dans des scénarios de suivi
    réels pour évaluer les performances du tracker. Vous pouvez mesurer des métriques telles que la précision du suivi, la robustesse aux
    variations, le taux de fausses correspondances, etc."""
    match_thresh: float = 0.9
    """Ce paramètre peut être utilisé pour filtrer les boîtes englobantes dont le rapport d'aspect est trop extrême. Par exemple, si vous
    vous attendez à ce que les objets aient généralement un rapport d'aspect proche de 1 (carrés ou presque carrés), vous pouvez choisir
    une valeur inférieure pour ignorer les boîtes englobantes excessivement allongées ou étirées."""
    aspect_ratio_thresh: float = 3.0
    """Cette valeur détermine la taille minimale que doit avoir une boîte englobante pour être prise en compte. Si vous savez que les objets
    d'intérêt ont une taille minimale, fixez ce paramètre en conséquence. Cela peut aider à éliminer les détections indésirables de petits
    objets ou de bruit."""
    min_box_area: float = 1.0
    mot20: bool = False

In [6]:
# converts Detections into format that can be consumed by match_detections_with_tracks function
def detections2boxes(detections: Detections) -> np.ndarray:
    return np.hstack((
        detections.xyxy,
        detections.confidence[:, np.newaxis]
    ))


# converts List[STrack] into format that can be consumed by match_detections_with_tracks function
def tracks2boxes(tracks: List[STrack]) -> np.ndarray:
    return np.array([
        track.tlbr
        for track
        in tracks
    ], dtype=float)


# matches our bounding boxes with predictions
def match_detections_with_tracks(
    detections: Detections,
    tracks: List[STrack]
) -> Detections:
    if not np.any(detections.xyxy) or len(tracks) == 0:
        return np.empty((0,))

    tracks_boxes = tracks2boxes(tracks=tracks)
    iou = box_iou_batch(tracks_boxes, detections.xyxy)
    track2detection = np.argmax(iou, axis=1)

    tracker_ids = [None] * len(detections)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            tracker_ids[detection_index] = tracks[tracker_index].track_id

    return tracker_ids

In [7]:
CLASSES_LIST = ["Swimming","Drowning"]

def predict_single_action(video_file_path, SEQUENCE_LENGTH):
    '''
    This function will perform single action recognition prediction on a video using the LRCN model.
    Args:
    video:  The video stream to perform action recognition on (an object created using cv2.VideoWriter).
    SEQUENCE_LENGTH:  The fixed number of frames of a video that can be passed to the model as one sequence.
    '''

    # Initialize the VideoCapture object to read from the video file.
    video_reader = cv2.VideoCapture(video_file_path)

    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Declare a list to store video frames we will extract.
    frames_list = []

    # Initialize a variable to store the predicted action being performed in the video.
    predicted_class_name = ''

    # Get the number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),0) #1

    # Iterating the number of times equal to the fixed length of sequence.
    for frame_counter in range(SEQUENCE_LENGTH):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Read a frame.
        success, frame = video_reader.read()

        # Check if frame is not read properly then break the loop.
        if not success:
            break

        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))

        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1.
        normalized_frame = resized_frame / 255

        # Appending the pre-processed frame into the frames list
        frames_list.append(normalized_frame)

    # Passing the  pre-processed frames to the model and get the predicted probabilities.
    predicted_labels_probabilities = convlstm_model.predict(np.expand_dims(frames_list, axis = 0), verbose=False)[0]

    # Get the index of class with highest probability.
    predicted_label = np.argmax(predicted_labels_probabilities)

    # Get the class name using the retrieved index.
    predicted_class_name = CLASSES_LIST[predicted_label]

    # Display the predicted action along with the prediction confidence.
    #print(f'Action Predicted: {predicted_class_name}\nConfidence: {predicted_labels_probabilities[predicted_label]}')
    return predicted_class_name,predicted_labels_probabilities[predicted_label]

In [8]:
video_path = "/content/drive/MyDrive/depositphotos_190694592-stock-video-holiday-beach-spain-summer-typical.mp4"
cap = cv2.VideoCapture(video_path)
#desired_fps = 3  # La fréquence à laquelle vous souhaitez lire les frames
#cap.set(cv2.CAP_PROP_FPS, desired_fps)
fps_ = int(cap.get(5))
print("fps:", fps_)
decalage = math.ceil(fps_ / 6)
print("decalage:", decalage)

output_directory_ = "/content/"
csv_file_path = "/content/zones_info.csv"

rows = 3
cols = 3
zones = rows * cols

model_path = 'yolov8x.pt'
conf_val = 0.28
#En ajustant le seuil d'IoU, vous pouvez contrôler la sensibilité de la suppression non maximale. Un seuil d'IoU plus élevé signifie que les boîtes doivent se
#chevaucher plus étroitement pour être supprimées, ce qui peut entraîner plus de détections conservées mais aussi des duplications. Un seuil plus bas peut être
#plus tolérant aux chevauchements partiels.
iou_vl = 0.4
agnostic_nms_vl = True

execution_time = 300
nb_frame = 0
var_value = {}

# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64

# Specify the number of frames of a video that will be fed to the model as one sequence.
SEQUENCE_LENGTH = 35

decale = 0

convlstm_model = load_model('/content/drive/MyDrive/model_con_lstm__model___Date_Time_2023_08_16__16_11_17___Loss_0.2769573926925659___Accuracy_0.9120879173278809.h5')

variables_trk = {}
for zone_id in range(zones):
  var_name = f"Tracks_zone_{zone_id}"
  byte_tracker = BYTETracker(BYTETrackerArgs())
  variables_trk[var_name] = byte_tracker

fps: 29
decalage: 5


In [ ]:
while True:
  success, frame = cap.read()

  if not success:
        break
  # Convertir le frame OpenCV en objet Image
  img = Image.fromarray(frame)
  if (nb_frame % decalage == 0):
    nb_frame += 1
    print(f'\n\nFrame N°{nb_frame} :')
    if (execution_time >= 300): # chaque 5 min
      execution_time = 0
      if(nb_frame > 1):
        # Ouvrir le fichier en mode écriture pour effacer le contenu
        with open(csv_file_path, 'w') as file:
            pass  # Cette instruction ne fait rien, ce qui effacera le contenu du fichier

      # Convertir le frame OpenCV en objet Image
      #img = Image.fromarray(frame)

      output_directory_ = "/content/"
      divide_image(img, rows=3, cols=3, output_directory=output_directory_)

      list_num_swimmers = []
      for row in range(rows):
        for col in range(cols):
          zone_image_path = f"/content/zone_{row}_{col}.png"
          num_swimmers = detect_objects_in_zone(zone_image_path, model_path, conf_val, output_directory_, iou = iou_vl,agnostic_nms = agnostic_nms_vl)
          list_num_swimmers.append(num_swimmers)

      df_zones = pd.read_csv(csv_file_path)
      # Add the 'num_swimmers' column to the DataFrame
      df_zones['num_swimmers'] = list_num_swimmers
      # Save the updated DataFrame back to the CSV file
      df_zones.to_csv(csv_file_path, index=False)

      zone_levl = 0
      zones_a_masque = []
      zones_non_masque = []
      for zone_id in range(zones):
        zone_levl += 1
        num_swimmers = df_zones.loc[zone_id, 'num_swimmers']
        if (zone_levl < 4 and num_swimmers > 8): # Changer la valeurs "num_swimmers" selon vos besoins
          zones_a_masque.append(zone_id)
        elif (zone_levl >= 4 and zone_levl < 7 and num_swimmers > 5):
          zones_a_masque.append(zone_id)
        elif (zone_levl >= 7 and zone_levl < 10 and num_swimmers > 2):
          zones_a_masque.append(zone_id)
        else :
          zones_non_masque.append(zone_id)
      # Créer un masque vide (tout en blanc)
      #mask = np.ones(img.shape, dtype=np.uint8) * 255

      # Boucle pour masquer chaque zone
      #for zone_id in zones_a_masque:
      #    x0 = df_zones.loc[zone_id, 'x0']
      #    y0 = df_zones.loc[zone_id, 'y0']
      #    x1 = df_zones.loc[zone_id, 'x1']
      #    y1 = df_zones.loc[zone_id, 'y1']

          # Créer un masque pour la zone à masquer (tout en noir)
      #    cv2.rectangle(mask, (x0, y0), (x1, y1), (0, 0, 0), -1)

    start_time = time.time()

    # Masquer les zones dans l'image
    #masked_image = cv2.bitwise_and(img, mask)

    for zone_id in zones_non_masque:
      print(f'\n\tZone N°{zone_id} :')
      x0 = df_zones.loc[zone_id, 'x0']
      y0 = df_zones.loc[zone_id, 'y0']
      x1 = df_zones.loc[zone_id, 'x1']
      y1 = df_zones.loc[zone_id, 'y1']

      # Extraire la zone de l'image
      zone_img = img.crop((x0, y0, x1, y1))

      # Create an instance of the YOLO model using the specified model path
      model = YOLO(model_path)

      # create BYTETracker instance
      desired_key = f"Tracks_zone_{zone_id}"
      byte_tracker = variables_trk[desired_key]

      # Convert the input image to a NumPy array in BGR format
      frame_np = cv2.cvtColor(np.array(zone_img), cv2.COLOR_RGB2BGR)

      results = model(frame_np, conf=conf_val, classes=0, iou = iou_vl, agnostic_nms = agnostic_nms_vl, show=False, verbose=False)

      # dict maping class_id to class_name
      CLASS_NAMES_DICT = model.model.names
      # class_ids of interest personne
      CLASS_ID = [0]

      detections = Detections(
          xyxy=results[0].boxes.xyxy.cpu().numpy(),
          confidence=results[0].boxes.conf.cpu().numpy(),
          class_id=results[0].boxes.cls.cpu().numpy().astype(int)
      )

      # filtering out detections with unwanted classes
      massk = np.array([class_id in CLASS_ID for class_id in detections.class_id], dtype=bool)
      detections=detections.filter(mask=massk, inplace=True)

      # tracking detections
      tracks = byte_tracker.update(
          output_results=detections2boxes(detections=detections),
          img_info=frame_np.shape,
          img_size=frame_np.shape
      )
      tracker_id = match_detections_with_tracks(detections=detections, tracks=tracks)
      detections.tracker_id = np.array(tracker_id)

      # filtering out detections without trackers
      maskk = np.array([tracker_id is not None for tracker_id in detections.tracker_id], dtype=bool)
      detections.filter(mask=maskk, inplace=True)
      if(len(detections)==0):
        print(f'\tNo détection.')
      for i in range(len(detections)):
        confidence = detections.confidence[i]  # Confiance de la détection
        confidence = math.ceil(confidence * 100)
        class_id = detections.class_id[i]  # ID de la classe de la détection
        tracker_id = detections.tracker_id[i]  # ID du tracker associé à la détection
        print(f'\tSwimmer Id {tracker_id} is detected.')
        dossier_nageur = f"/content/zon_id_{zone_id}_tracker_id_{tracker_id}"
        os.makedirs(dossier_nageur, exist_ok=True)
        bbox = detections.xyxy[i]  # Coordonnées de la boîte englobante au format xyxy
        x_min = bbox[0]
        y_min = bbox[1]
        x_max = bbox[2]
        y_max = bbox[3]
        x_min, y_min, x_max, y_max = int(x_min), int(y_min), int(x_max), int(y_max)
        width = x_max - x_min
        height = y_max - y_min
        x1, y1, x2, y2 = x_min-(width*0.7), y_min-(height*0.7), x_max+(width*0.7), y_max+(height*0.7)
        cropped_personne = zone_img.crop((x1, y1, x2, y2))
        image_path_ = os.path.join(dossier_nageur, f"frame_{int(nb_frame)}.png")
        cropped_personne_array = np.array(cropped_personne)
        # Resize the image using the cv2.resize function
        resized_image = cv2.resize(cropped_personne_array, (IMAGE_HEIGHT, IMAGE_WIDTH))#########
        cv2.imwrite(image_path_, resized_image)
        #print(f"Detection {i}: Bbox: {bbox}, Confidence: {confidence}, Class ID: {class_id}, Tracker ID: {tracker_id}")

        # Get a list of all files in the folder
        all_files = os.listdir(dossier_nageur)
        # Filter the list to include only image files
        image_files = [filename for filename in all_files if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
        # Get the number of image files
        num_images = len(image_files)

        if(num_images > (35)) : # 30 frames/s pendant 20s (+ decale)
          #print("Generer un video!!!!!!")
          #decale += 42 # 30 frames/s pendant 5s
          dossier_nageur = f"/content/zon_id_{zone_id}_tracker_id_{tracker_id}"

          # Liste des noms de fichiers d'images dans le dossier
          images = [f for f in os.listdir(dossier_nageur) if f.endswith('.png')]

          # Trier les noms de fichiers en fonction des numéros de frame
          images.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))
          # Prendre seulement les 600 dernières images
          images = images[-35:]

          # Obtenir les dimensions de la première image (elles seront utilisées pour la vidéo)
          premiere_image = cv2.imread(os.path.join(dossier_nageur, images[0]))
          hauteur, largeur, _ = premiere_image.shape
          image_resolution = (largeur, hauteur)

          # Initialiser le writer vidéo en mémoire
          output_filename = os.path.join(dossier_nageur, "output.mp4")
          if os.path.exists(output_filename):
            os.remove(output_filename)
          fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Vous pouvez également utiliser d'autres codecs (*'XVID')
          fps = 5
          out = cv2.VideoWriter(output_filename, fourcc, fps, image_resolution)

          # Lire et écrire chaque image dans la vidéo
          for image in images:
              image_path = os.path.join(dossier_nageur, image)
              frame = cv2.imread(image_path)
              out.write(frame)
          # Libérer les ressources
          out.release()
          predicted_class_name ,Confidence= predict_single_action(output_filename, SEQUENCE_LENGTH)
          Confidence = math.ceil(Confidence * 100)
          if(predicted_class_name == "Swimming"):
            message = f"\tSwimmer with ID {tracker_id} is currently swimming in Zone {zone_id} with a confidence level of {Confidence}% :)."
            message_vert = "\033[92m" + message + "\033[0m"
            print(message_vert)
          else:
            message = f'\tSwimmer with ID {tracker_id} is currently drowning in Zone {zone_id} with a confidence level of {Confidence}% !!!!!!!'
            message_rouge = "\033[91m" + message + "\033[0m"
            print(message_rouge)

    end_time = time.time()
    execution_time += end_time - start_time
  else:
    nb_frame += 1



Frame N°1 :


100%|██████████| 131M/131M [00:01<00:00, 111MB/s]



	Zone N°0 :
	No détection.

	Zone N°1 :
	No détection.

	Zone N°2 :
	Swimmer Id 1 is detected.
	Swimmer Id 2 is detected.


Frame N°6 :

	Zone N°0 :
	No détection.

	Zone N°1 :
	No détection.

	Zone N°2 :
	Swimmer Id 1 is detected.
	Swimmer Id 2 is detected.


Frame N°11 :

	Zone N°0 :
	No détection.

	Zone N°1 :
	No détection.

	Zone N°2 :
	Swimmer Id 1 is detected.
	Swimmer Id 2 is detected.


Frame N°16 :

	Zone N°0 :
	No détection.

	Zone N°1 :
	No détection.

	Zone N°2 :
	Swimmer Id 1 is detected.
	Swimmer Id 2 is detected.


Frame N°21 :

	Zone N°0 :
	No détection.

	Zone N°1 :
	No détection.

	Zone N°2 :
	Swimmer Id 2 is detected.
	Swimmer Id 1 is detected.


Frame N°26 :

	Zone N°0 :
	No détection.

	Zone N°1 :
	No détection.

	Zone N°2 :
	Swimmer Id 1 is detected.


Frame N°31 :

	Zone N°0 :
	No détection.

	Zone N°1 :
	No détection.

	Zone N°2 :
	Swimmer Id 1 is detected.


Frame N°36 :

	Zone N°0 :
	No détection.

	Zone N°1 :
	No détection.

	Zone N°2 :
	Swimmer Id 1 is det